Wiki Scraper
---

Author: Peter Zhang

Scraping tool for the Wiki.

### Setup

#### Imports

In [1]:
# imports
import urllib.request, urllib.parse, urllib.error
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os.path
from os import path
import re

#### Settings

- OVERWRITE determines whether or not to update existing files.
- PAGES_URL is a list of Wiki pages
- OUTPATH is where files are stored

In [19]:
# settings
OVERWRITE = True
TARGET_WIKIS = ["CX16", "CX17", "CX18", "CX19", "CX20"]

In [20]:
# get page URLs
WIKIS_URL = "tools/wiki_pages.csv"
WIKIS = [row for row in csv.reader(open(WIKIS_URL, 'r'))]

In [21]:
# outpath
OUTPATH = "wiki_data/"

### Scrapers

#### Wiki Page

Take the URL to a wiki page and returns a list of all the school names and their respective URLs.

In [22]:
# for a given archive year, return all schools
def getSchools(url):
    
    # try opening URL
    try:
        html = urlopen(url).read()
    except Exeption:
        print("Wiki URL broke")
        return []
    
    
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    # find all links
    links = soup.find_all('a')
    
    # find only school links
    links = [link for link in links if "(" in link.text and ")" in link.text]
    
    # get link text and URL
    schools = [[link.text, link.get('href')] for link in links]

    return schools

In [23]:
getSchools("https://hspolicy19.debatecoaches.org/")

[['Advanced Technologies Academy (NV)', '/Advanced%20Technologies%20Academy/'],
 ['Airline (AL)', '/Airline/'],
 ['Aledo (TX)', '/Aledo/'],
 ['Alief Taylor (TX)', '/Alief%20Taylor/'],
 ['Alpharetta (GA)', '/Alpharetta/'],
 ['Altamont (AL)', '/Altamont/'],
 ['American Heritage (FL)', '/American%20Heritage/'],
 ['Anderson (TX)', '/Anderson/'],
 ['Andover (KS)', '/Andover/'],
 ['Andover Central (KS)', '/Andover%20Central/'],
 ['Andrews (TX)', '/Andrews/'],
 ['Archbishop Mitty (CA)', '/Archbishop%20Mitty/'],
 ['Ashland (OR)', '/Ashland/'],
 ['Asian Debate League (AK)', '/Asian%20Debate%20League/'],
 ['Athens (TX)', '/Athens/'],
 ['Aubrey (TX)', '/Aubrey/'],
 ['BASIS Chandler (AZ)', '/BASIS%20Chandler/'],
 ['BASIS Peoria (AZ)', '/BASIS%20Peoria/'],
 ['BASIS Shavano (TX)', '/BASIS%20Shavano/'],
 ['Bakersfield (CA)', '/Bakersfield/'],
 ['Baltimore City College (MD)', '/Baltimore%20City%20College/'],
 ['Barstow (MO)', '/Barstow/'],
 ['Baton Rouge Magnet (LA)', '/Baton%20Rouge%20Magnet/'],
 ['B

#### School Page

Take the URL to a school and return a list of debaters and the URLs that point to their Aff and Neg pages

In [24]:
# return aff and neg URLs for all the debaters of a given school

def getTeams(url):
    
    # try opening URL
    try:
        html = urlopen(url).read()
    except Exception:
        return (0, [], {}, "School URL broke")
        
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    # look for a table
    if len(soup.find_all("table")):
        table = soup.find_all("table")[0]
    else:
        return (0, [], {}, "School page not set up")
        
    # exclude header
    numDebaters = len(table.find_all('tr'))-1
    
    if numDebaters == 0:
        return (0, [], {}, "No debaters listed")
    
    # track team names and map to URLs
    teamNames = []
    pageURLs = {}
    
    
    # for each table row
    for row in table.find_all('tr')[1:]:
        
        # the first column's string is the team name
        team = row.find('td').string
        teamNames.append(team)
        
        # collect the URLs to the aff and neg page
        links = row.find_all('a')
        URLs = [link.get("href") for link in links]
        pageURLs[team] = URLs
    
    return (numDebaters, teamNames, pageURLs, "")


In [25]:
getTeams("https://hspolicy19.debatecoaches.org/Advanced%20Technologies%20Academy")

(3,
 ['Advanced Technologies Academy Jacqueline Balanovsky - Raymond Behnke',
  'Advanced Technologies Academy Sam Self - Jonah Gentleman',
  'Advanced Technologies Academy Ryan Fritchel - Hannah Lewis'],
 {'Advanced Technologies Academy Jacqueline Balanovsky - Raymond Behnke': ['/Advanced%20Technologies%20Academy/Balanovsky-Behnke%20Aff',
   '/Advanced%20Technologies%20Academy/Balanovsky-Behnke%20Neg'],
  'Advanced Technologies Academy Sam Self - Jonah Gentleman': ['/Advanced%20Technologies%20Academy/Self-Gentleman%20Aff',
   '/Advanced%20Technologies%20Academy/Self-Gentleman%20Neg'],
  'Advanced Technologies Academy Ryan Fritchel - Hannah Lewis': ['/Advanced%20Technologies%20Academy/Fritchel-Lewis%20Aff',
   '/Advanced%20Technologies%20Academy/Fritchel-Lewis%20Neg']},
 '')

#### Team Page

In [26]:
def getDisclosure(url):

    # try opening URL
    try:
        html = urlopen(url).read().decode('utf-8-sig')
        
    except Exception:
        return {"Team URL":url,
                "Rounds": [],
                "Round Reports": [],
                "Cites": [],
                "Errors": "Did not open"}
    
    
    # soupify
    soup = BeautifulSoup(html, "html.parser")
    
    tables = soup.find_all("table")

    if len(tables) > 2:
        
        rnds = tables[0]
        rndrprts = tables[1]
        cts = tables[2]
    
    else:
        
        return {"Team URL":url,
                "Rounds": [],
                "Round Reports": [],
                "Cites": [],
                "Errors": "Not configured properly"}
    
    # collect all rounds
    rounds = []
    
    for row in rnds.find_all("tr")[1:]:
        
        # get pieces
        cols = row.find_all("td")
        trn = cols[0].text
        rnd = cols[1].text
        opp = cols[2].text
        jdg = cols[3].text
        osrc = cols[6].find("a")
        if osrc:
            docURL = osrc.get("href")
        else:
            docURL = ""
            
        rounds.append([trn, rnd, opp, jdg, docURL])
    
    # collect all round reports
    roundReports = []
    
    for row in rndrprts.find_all("tr")[1:]:
        try:
            pars = roundReports.append(row.find_all("td")[2].find_all("p")[1].text)
        except:
            continue

    # collect all cites
    cites = [row.find_all("td")[0].find("span").text for row in cts.find_all('tr')[1:]]
        
    return {"Team URL":url,
            "Rounds": rounds,
            "Round Reports": roundReports,
            "Cites": cites,
            "Errors": ""}
    


In [27]:
getDisclosure("https://hsld17.debatecoaches.org/Acton-Boxborough/Liu%20Aff")

{'Team URL': 'https://hsld17.debatecoaches.org/Acton-Boxborough/Liu%20Aff',
 'Rounds': [['-', 'Finals', '-', '-', ''],
  ['Harvard', '2', 'Dan Shahab', 'David Moon', ''],
  ['Lexington Winter Invitational', '2', 'Scarsdale AW', 'David Moon', ''],
  ['Newark Invitational', '4', 'Stuyvesant JL', 'Amit Kukreja', ''],
  ['Practice Round', '1', 'Michelle I drop T Li', 'the coolest judge', ''],
  ['Practice Round', 'Semis', '-', '-', '']],
 'Round Reports': [],
 'Cites': ['Contact',
  'JF - Brown Bear AC',
  'JF - Brown Bear AC v2',
  'JF - Brown Bear AC v3',
  'JF - Deleuzian Narrativity 1AC'],
 'Errors': ''}

#### Execution

Loop through wikis, find all schools, visit all entry pages, scrape all positions.

In [ ]:
for wiki in WIKIS:
    
    # unpack wiki
    wikiName = wiki[0]
    wikiURL = wiki[1]
    
    # if not in target wikis, don't write
    if wikiName not in TARGET_WIKIS:
        continue
    
    # if it exists, don't overwrite
    if (not OVERWRITE) and path.exists(OUTPATH + wikiName + "_schools_wiki.csv"):
        continue
        
    # open output file
    with open(OUTPATH + wikiName + "schools_wiki.csv", 'w', encoding="utf-8") as SCHOOLFILE, open(OUTPATH + wikiName + "teams_wiki.csv", 'w', encoding="utf-8") as TEAMFILE:
        
        # instantaite writer
        schoolWriter = csv.DictWriter(SCHOOLFILE,
                                     fieldnames = ["School Name",
                                                   "School URL",
                                                   "School Teams",
                                                   "Errors"],
                                      quotechar='"', 
                                      quoting=csv.QUOTE_NONNUMERIC,
                                     lineterminator = "\n")
        
        # write header
        schoolWriter.writeheader()
        
        # instantaite writer
        teamWriter = csv.DictWriter(TEAMFILE,
                                    fieldnames = ["Team Name",
                                                  "Side",
                                                  "Team URL",
                                                  "Rounds",
                                                  "Round Reports",
                                                  "Cites",
                                                  "Errors"],
                                    lineterminator = "\n")
        
        # write header
        teamWriter.writeheader()
            
        print("Scraping " + wikiName)
        
        # get school URLs
        schools = getSchools(wikiURL)
        
        print("There are " + str(len(schools)) + " schools")
        
        for school in schools:
            
            # get school name and URL
            schoolName, schoolURL = school[0], wikiURL + school[1]
            
            print("Checking " + schoolName)
            
            # get teams from the school
            numTeams, teamNames, teamURLs, errors = getTeams(schoolURL)
            
            print("Found " + str(numTeams) + " teams")
            
            # write school information
            schoolWriter.writerow({"School Name" : schoolName,
                                  "School URL": schoolURL,
                                  "School Teams": teamNames,
                                  "Errors": errors})
            
            for team in teamNames:
                
                try:
                
                    print("Scraping " + team)

                    # get team URL
                    teamURL = teamURLs[team]
                    affURL = wikiURL + teamURL[0]
                    negURL = wikiURL + teamURL[1]

                    # aff scraping
                    affInfo = getDisclosure(affURL)
                    affInfo["Team Name"] = team
                    affInfo["Side"] = "Aff"

                    # write aff info
                    teamWriter.writerow(affInfo)

                    # neg scraping
                    negInfo = getDisclosure(negURL)
                    negInfo["Team Name"] = team
                    negInfo["Side"] = "Neg"

                    # write neg info
                    teamWriter.writerow(negInfo)
                    
                except:
                    
                    print(team + " failed.")
                    
                

Scraping CX16
There are 527 schools
Checking AM Consolidated (TX)
Found 0 teams
Checking Abernathy (TX)
Found 0 teams
Checking ACORN Community (NY)
Found 0 teams
Checking Agape Leaders Prep (NY)
Found 0 teams
Checking Airline (LA)
Found 2 teams
Scraping Airline Amanda Watson - Miranda Weathersby
Scraping Airline Nandini Tivakaran - Chloe Brown
Checking Albuquerque Academy (NM)
Found 5 teams
Scraping Albuquerque Academy Rafael Schleder - Calvin Abdallah
Scraping Albuquerque Academy Akshay Warrier - Kevin Xue
Scraping Albuquerque Academy Gabriel Sandoval - Nikita Jaiswal
Scraping Albuquerque Academy Jeremy Rifkin - Sage Min
Scraping Albuquerque Academy Jeremy Rifkin - Akshay Warrier
Checking Alpharetta (GA)
Found 10 teams
Scraping Alpharetta Isha Deshmukh - Shloka Danave
Scraping Alpharetta Pranav Mutyala - Adam Lederer
Scraping Alpharetta Shreya Sachdeva - Pooja Udeshi
Scraping Alpharetta Siddharth Madhavan - Adam Lederer
Scraping Alpharetta Priyanka Gokare - Anav Mittal
Scraping Alphar

Scraping Blue Valley Southwest Anna Lee - Ted Shi
Scraping Blue Valley Southwest Chuck Castle - Rachel Holzer
Scraping Blue Valley Southwest Max McCarty - Eli Pieper
Checking Blue Valley West (KS)
Found 4 teams
Scraping Blue Valley West Seena Saiedian - Animesh Shrouti
Scraping Blue Valley West Branden Kohnle - Evan Sander
Scraping Blue Valley West Daniel Birzer - Srivats Narayanan
Scraping Blue Valley West Dan Vasudevan - Jack Schrag
Checking Bonneville (ID)
Found 0 teams
Checking Boston Debate League (MA)
Found 2 teams
Scraping Boston Debate League Roger Nix - Rane Baldwin
Scraping Boston Debate League Brooke Modestita - Zaiana Falcon
Checking Boston Latin (MA)
Found 1 teams
Scraping Boston Latin Mosie Burke - Maryanne Pasiewicz
Checking Bravo Medical (CA)
Found 2 teams
Scraping Bravo Medical Jumico Evi - Harut Kejejyan
Scraping Bravo Medical Arman Kosoyan - Karla Cruz
Checking Brentwood (CA)
Found 1 teams
Scraping Brentwood Whit Jackson - Emma Blum
Checking Briar Woods (VA)
Found 0 

Checking Chesterton (IN)
Found 0 teams
Checking CK McClatchy (CA)
Found 8 teams
Scraping CK McClatchy Annabelle Long - Elyse Pham
Scraping CK McClatchy Abigail Norton - Molly Martin
Scraping CK McClatchy Emily Valerio - Gabriel Ketron
Scraping CK McClatchy Adrienne Marcin - Olivia AngOlson
Scraping CK McClatchy Arin Wise - Ian Keithley
Scraping CK McClatchy Nick Fleming - Maya Steinhart
Scraping CK McClatchy Libby Reynolds - Abby Gray
Scraping CK McClatchy Molly Martin - Devin Anderson
Checking Clackamas (OR)
Found 0 teams
Checking Claremont (CA)
Found 0 teams
Checking Classical Academy (CA)
Found 5 teams
Scraping Classical Academy John Melton - Jake Melton
Scraping Classical Academy Crystal Sung - Christopher Tai
Scraping Classical Academy Alex Tseng - Christian Worthen
Scraping Classical Academy Alex Tseng - Christopher Tai
Scraping Classical Academy Daniel Yu - David Zhou
Checking Classical Davies (RI)
Found 0 teams
Checking Clear Lake (TX)
Found 2 teams
Scraping Clear Lake Haris Ay

Scraping Downtown Magnets Itzayana Lopez - Lissette Gomez
Scraping Downtown Magnets Alfonso Santos - Yardley Rosas
Scraping Downtown Magnets Wilmer Donis - Christopher Perez
Scraping Downtown Magnets Joshua Sto Domingo - Jewell Graham
Scraping Downtown Magnets Christopher Perez - Camille Grundy
Checking Dripping Springs (TX)
Found 0 teams
Checking Dulles (TX)
Found 5 teams
Scraping Dulles Angelina Zhang - Anusha Rajwani
Scraping Dulles Elijah Valerio - Timothy Yen
Scraping Dulles Anusha Rajwani - Timothy Yen
Scraping Dulles Angelina Zhang - Elijah Valerio
Scraping Dulles Hanzheng Li - Ryan Chu
Checking Dunwoody (GA)
Found 0 teams
Checking Eagan (MN)
Found 3 teams
Scraping Eagan Maddie Roberts - Brandon Wilary
Scraping Eagan Lizzy Sabel - Linnea Stanton
Scraping Eagan Greg Quick - Aaron Lutz
Checking Eagle (ID)
Found 2 teams
Scraping Eagle Michael Eusterman - Mark Eusterman
Scraping Eagle Carli Buzzard - Charlee Wambolt
Checking East (UT)
Found 2 teams
Scraping East Willis Holle - Lucas

Scraping Green Valley Alex Safieddine - Anthony Winchell
Scraping Green Valley Rushil Shah - Anthony Winchell
Scraping Green Valley Reema Amhaz - Alex Safieddine
Checking Greenhill (TX)
Found 14 teams
Scraping Greenhill Sidarth Subramanian - Grace Kuang
Scraping Greenhill Diego Marrero - Ali Khambati
Scraping Greenhill Sam Grimsley - Chris Eckert
Scraping Greenhill Samuel Charnes - Shaurya Tathgir
Scraping Greenhill Timothy Wegener - Samar Ahmad
Scraping Greenhill Josh Leffler - Ryan Zlotzky
Scraping Greenhill Swati Ravi - Christine Wu
Scraping Greenhill Sam Grimsley - Swati Ravi
Scraping Greenhill Ethan Rothstein - Josh Kwon
Scraping Greenhill Nicholas Mui - Arya Nallanthighall
Scraping Greenhill Christine Wu - Reid Zlotky
Scraping Greenhill Anais Zhang - Sophia Hurst
Scraping Greenhill Josh Kwon - Reid Zlotky
Scraping Greenhill Chris Eckert - Timothy Wegener
Checking Greenwood (AR)
Found 0 teams
Checking Greenwood Lab (MO)
Found 0 teams
Checking Groves (MI)
Found 4 teams
Scraping Gro

...

Scraping Bingham Mia Orton - Nadia Sabry
Checking Bishop Guertin (NH)
Found 0 teams
Checking Bishop Kelly (ID)
Found 0 teams
Checking Bishop Loughlin (NY)
Found 0 teams
Checking Bishop O'Dowd (CA)
Found 0 teams
Checking Blaine (MN)
Found 0 teams
Checking Blake (MN)
Found 2 teams
Scraping Blake Allen Cao - Cole Mathews
Scraping Blake Kennedy Rupert - Flora Yang
Checking Bloomington (MN)
Found 0 teams
Checking Blue Springs (MO)
Found 0 teams
Checking Blue Valley North (KS)
Found 8 teams
Scraping Blue Valley North Max Indiveri - Chris Lucas
Scraping Blue Valley North Vamsi Chavali - Eleanor Mendelson
Scraping Blue Valley North Ramya Chilappa - Ellie Anderson
Scraping Blue Valley North Mickey McMahon - Avanyish Toniappa
Scraping Blue Valley North Vamsi Chavali - Avanyish Toniappa
Scraping Blue Valley North Mickey McMahon - Eleanor Mendelson
Scraping Blue Valley North Max Indiveri - Sneha Kannan
Scraping Blue Valley North Ellie Anderson - Chris Lucas
Checking Blue Valley Northwest (KS)
Foun

Scraping Carrollton Sacred Heart Dorothy Darden - Mariana Carta
Scraping Carrollton Sacred Heart Gabriela Garity - Lily Touret
Scraping Carrollton Sacred Heart Caroline Culmo - Cristina Jugo
Scraping Carrollton Sacred Heart Carmen AliaArias - Carolina Perez
Scraping Carrollton Sacred Heart Gabriela Garity - Alexa De Paz
Scraping Carrollton Sacred Heart Caroline Culmo - Gabriela Garity
Scraping Carrollton Sacred Heart Sabrina Chamizo - Natalie Giustini
Scraping Carrollton Sacred Heart Novices Unknown - Other Novices
Scraping Carrollton Sacred Heart Camille Deschapelles - Emma Beharry
Checking Casady (OK)
Found 4 teams
Scraping Casady Armaan Aggarwal - Kunal Patel
Scraping Casady Malavika Rajaram - Cooper Lindstrom
Scraping Casady Armaan Aggarwal - Cooper Lindstrom
Scraping Casady Teams All - All Teams
Checking Cascia Hall (OK)
Found 0 teams
Checking Cathedral Prep (PA)
Found 2 teams
Scraping Cathedral Prep Daanish Bhatti - Justin Czerwinski
Scraping Cathedral Prep Noah Ali - Owen Jacobs

Checking Damien (CA)
Found 5 teams
Scraping Damien Tristan Bato - Dylan Johnson
Scraping Damien Joseph Banuelos - Leon Garcia
Scraping Damien Nolan DeJesus - Brendan Tremblay
Scraping Damien Tristan Bato - Brendan Tremblay
Scraping Damien Dylan Johnson - Colin Coppock
Checking Davis (CA)
Found 2 teams
Scraping Davis Jack Walsh - Amanda Niemela
Scraping Davis Lili Ma - Max Komessar
Checking Dawson (TX)
Found 0 teams
Checking Debate Rhode Island (RI)
Found 0 teams
Checking Denver Arts (CO)
Found 0 teams
Checking Denver Center For Int'l Studies (CO)
Found 5 teams
Scraping Denver Center Kathleen Shea - Kim Nguyen
Scraping Denver Center Hannah Chapman - Carlotta Jumpertz
Scraping Denver Center Angelica Cerna - Max Fireman
Scraping Denver Center Owen Schalk - Reyna Esquibel
Scraping Denver Center Madison Alexander - Mei Qi Liang
Checking Denver East (CO)
Found 2 teams
Scraping Denver East Acacia Downs - Sophie Koeing
Scraping Denver East Claire Ely - Ava Barish
Checking Derby (KS)
Found 5 te

Scraping Fort Lauderdale Karin Rashid - Angie Vassallo
Scraping Fort Lauderdale Kent Raticoff - Dillon Farrington
Checking Fort Osage (MO)
Found 0 teams
Checking Fremont (NE)
Found 0 teams
Checking Friendswood (TX)
Found 0 teams
Checking Fullerton Union (CA)
Found 1 teams
Scraping Fullerton Union Daniel Gray - Ryan Koh
Checking Gabrielino (CA)
Found 1 teams
Scraping Gabrielino Michael Chiu - Michael Hong
Checking Galloway (GA)
Found 0 teams
Checking Garfield (WA)
Found 5 teams
Scraping Garfield Hannah Christianson - Petra Zuniga
Scraping Garfield Carter RasnicOlson - Luke Porter
Scraping Garfield Carter Rasnic Olson - Stella Robertson
Scraping Garfield Don Draper - Rhys Lawson
Scraping Garfield Mabel Baumgardner - Luke Porter
Checking George Walton (GA)
Found 0 teams
Checking George Washington (CO)
Found 0 teams
Checking Georgetown Day (DC)
Found 12 teams
Scraping Georgetown Day Thomas Brooks - Robin Forsyth
Scraping Georgetown Day Tara Bhagat - Alan Goldfarb
Scraping Georgetown Day Th

Scraping Harker Karoun Kaushik - Deeya Viradia
Scraping Harker Deven Parikh - Mir Bahri
Scraping Harker Andy Lee - Deven Shah
Scraping Harker Deven Shah - Mir Bahri
Scraping Harker Saanvi Arora - Sarah Mohammed
Checking Harlingen South (TX)
Found 0 teams
Checking Harrisonburg (VA)
Found 0 teams
Checking Harvard-Westlake (CA)
Found 3 teams
Scraping Harvard-Westlake Jake Davidson - Spencer Paul
Scraping Harvard-Westlake Jaya Nayar - Alexandra Mork
Scraping Harvard-Westlake Jake Davidson - Vishan Chaudhary
Checking Hawken (OH)
Found 1 teams
Scraping Hawken Ziqi Wei - Steven Sun
Checking Head Royce (CA)
Found 8 teams
Scraping Head Royce Larry Dang - Josh Hui
Scraping Head Royce Ty Navarro - Ian Lee
Scraping Head Royce Ty Navarro - Aidan Wogan
Scraping Head Royce Ty Navarro - Vikram Ramaswamy
Scraping Head Royce Ian Lee - Aidan Wogan
Scraping Head Royce Vikram Ramaswamy - Lisa Kopelnik
Scraping Head Royce Ty Navarro - Lisa Kopelnik
Scraping Head Royce Ian Lee - Lisa Kopelnik
Checking Heartl